# Tool error handling

Using a model to invoke a tool has some obvious potential failure modes. Firstly, the model needs to return a output that can be parsed at all. Secondly, the model needs to return tool arguments that are valid.

We can build error handling into our chains to mitigate these failure modes.

## Setup

We'll need to install the following packages:

In [ ]:
%pip install --upgrade --quiet langchain langchain-openai

And set these environment variables:

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# If you'd like to use LangSmith, uncomment the below:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Chain

Suppose we have the following (dummy) tool and tool-calling chain. We'll make our tool intentionally convoluted to try and trip up the model.

In [57]:
from operator import itemgetter

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function
from langchain_core.runnables import Runnable
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI


@tool
def complex_tool(int_arg: int, float_arg: float, dict_arg: dict) -> int:
    """Do something complex with a complex tool."""
    return int_arg * float_arg


tools = [complex_tool]
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)


def tool_chain(function_api_output: dict) -> Runnable:
    tool_map = {tool.name: tool for tool in tools}
    chosen = tool_map[function_api_output["name"]]
    return itemgetter("arguments") | chosen


chain = model | JsonOutputFunctionsParser(args_only=False) | tool_chain

In [58]:
chain.invoke("use tool on 5, 2.1, {}")

ValidationError: 1 validation error for complex_toolSchemaSchema
dict_arg
  field required (type=value_error.missing)

## Try/except tool call

The simplest way to more gracefully handle errors is to try/except the tool-calling step and return a helpful message on errors:

In [59]:
from typing import Any


def call_tool(function_api_output: dict, config: dict) -> Any:
    tool_map = {tool.name: tool for tool in tools}
    chosen = tool_map[function_api_output["name"]]
    try:
        return chosen.invoke(function_api_output["arguments"], config=config)
    except Exception as e:
        return f"Calling tool `{function_api_output['name']}` with arguments:\n\n{function_api_output['arguments']}\n\nraised the following error:\n\n{type(e)}: {e}"


chain = model | JsonOutputFunctionsParser(args_only=False) | call_tool

In [60]:
print(chain.invoke("use tool on 5, 2.1, {}"))

Calling tool `complex_tool` with arguments:

{'int_arg': 5, 'float_arg': 2.1}

raised the following error:

<class 'pydantic.v1.error_wrappers.ValidationError'>: 1 validation error for complex_toolSchemaSchema
dict_arg
  field required (type=value_error.missing)


## Retry with exception

To take things one step further, we can try to automatically re-run the chain with the exception passed in, so that the model may be able to correct its behavior:

In [83]:
import json
from typing import Any

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough


class CustomToolException(Exception):
    """Custom LangChain tool exception."""

    def __init__(
        self, tool_name: str, tool_arguments: dict, base_exception: Exception
    ) -> None:
        super().__init__()
        self.tool_name = tool_name
        self.tool_arguments = tool_arguments
        self.base_exception = base_exception


def call_tool(function_api_output: dict, config: dict) -> Any:
    tool_map = {tool.name: tool for tool in tools}
    chosen = tool_map[function_api_output["name"]]
    try:
        return chosen.invoke(function_api_output["arguments"], config=config)
    except Exception as e:
        return CustomToolException(
            tool_name=function_api_output["name"],
            tool_arguments=function_api_output["arguments"],
            base_exception=e,
        )


def retry_if_exception(inputs: dict, config: dict) -> Any:
    last_output = inputs["last_output"]
    if isinstance(last_output, CustomToolException):
        function_call = {
            "name": last_output.tool_name,
            "arguments": json.dumps(last_output.tool_arguments),
        }
        messages = [
            AIMessage(content="", additional_kwargs={"function_call": function_call}),
            HumanMessage(
                content=f"The last tool call raised exception:\n\n{inputs['last_output'].base_exception}\n\nTry calling the tool again with corrected arguments."
            ),
        ]
        return RunnablePassthrough.assign(last_output=lambda x: messages) | chain
    else:
        return inputs["last_output"]


prompt = ChatPromptTemplate.from_messages(
    [("human", "{input}"), MessagesPlaceholder("last_output", optional=True)]
)
chain = prompt | model | JsonOutputFunctionsParser(args_only=False) | call_tool
chain_with_retry = RunnablePassthrough.assign(last_output=chain) | retry_if_exception

In [85]:
chain_with_retry.invoke({"input": "use tool on 5, 2.1, empty dict"})

10.5